In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random

from torch.utils.tensorboard import SummaryWriter

In [ ]:
! wget https://www.dropbox.com/s/mw6ctze0ikcuqt4/mnist_train.csv?dl=0 -O mnist_train.csv

In [ ]:
! wget https://www.dropbox.com/s/e8n5v1sea2housp/mnist_test.csv?dl=0 -O mnist_test.csv

In [ ]:
train_data = np.loadtxt("mnist_train.csv", 
                        delimiter=",")
test_data = np.loadtxt("mnist_test.csv", 
                       delimiter=",")

In [ ]:
train_data.shape

In [ ]:
test_labels = test_data[:, 0].astype(int)
test_images = test_data[:, 1:].reshape(-1, 28*28) / 255.
train_labels = train_data[:, 0].astype(int)
train_images = train_data[:, 1:].reshape(-1, 28*28) / 255.

In [ ]:
ALPHA = 0.001
NUM_EPOCHS = 20
BATCH_SIZE = 32

# Dataset

In [ ]:
class MNIST_Trainset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
MNIST_loader = DataLoader(MNIST_Trainset(X=train_images, y=train_labels), batch_size=BATCH_SIZE, shuffle=True)

# Model

# Sequenziale

In [ ]:
model = nn.Sequential(
    nn.Linear(784, 50),
    nn.ReLU(),
    nn.Linear(50, 50),
    nn.ReLU(),
    nn.Linear(50, 10)
)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
EPOCHS = 100
LR = 1e-3
BATCH_SIZE = 32

In [ ]:
summary_writer = SummaryWriter()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
counter = 0
for epoch in range(EPOCHS):
    for i, (x_b, y_b) in enumerate(MNIST_loader):
        y_ = model(x_b.float())
        loss = F.cross_entropy(y_, y_b.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        counter += 1
        if counter % 100 == 0:
            y_test_ = model(torch.tensor(test_images).float())
            pred_test = torch.argmax(y_test_, dim=1)
            test_accuracy = accuracy_score(test_labels, pred_test)
            summary_writer.add_scalar('test accuracy', test_accuracy, counter)
        summary_writer.add_scalar('training loss', loss, counter)

# Accuratezza sul test set

In [ ]:
y_test_ = model(torch.tensor(test_images).float())

In [ ]:
pred_test = torch.argmax(y_test_, dim=1)

In [ ]:
from sklearn.metrics import  accuracy_score

In [ ]:
accuracy_score(test_labels, pred_test)

# Generico

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        pass
        
    def forward(self, x):
        return F.softmax('last layer', dim=1)

# Addestramento